# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Question 3.1

$(Aw)_t = w_1^T\tilde x(t)+w_0-y(t)w_2^T\tilde x(t)$ ; $b_t=y(t)$\
Suppose $Aw = b$ then $b_t = y(t) = w_1^T\tilde x(t)+w_0-y(t)w_2^T\tilde x(t) \Leftrightarrow y(t)(1+w_2^T\tilde x(t)) = w_1^T\tilde x(t)+w_0 \Leftrightarrow y(t) = \frac {w_1^T\tilde x(t)+w_0}{1+w_2^T\tilde x(t)}$ 

In [2]:
# Code from data_center_helper.py


data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

# Constructing matrices for min_w ||A w - b||_2**2

matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]
print(b.shape)

# Constructing matrices for the test set

M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]
print(b_test.shape)


# Loading raw data
data = pd.read_csv('Raw_Dataset_May.csv')
print(data.head())

def name_to_subcategory_and_details(col_name):
    if np.isreal(col_name):
        col_name = names[col_name]
    indices = np.nonzero((data['NAME'] == col_name).values)[0]
    if len(indices) > 0:
        subcategory = data['SUBCATEGORY'].iloc[[indices[0]]].values[0]
        details = data['DETAILS'].iloc[[indices[0]]].values[0]
        return subcategory, details
    else:
        print('unknown name')

(722,)
(361,)
   Unnamed: 0                              TIME     VALUE BUILDING  \
0           0  2022-05-01 19:24:38.178000+00:00  348307.0    bat01   
1           1  2022-05-02 16:04:38.148000+00:00  348497.0    bat01   
2           2  2022-05-30 05:24:37.188000+00:00  353967.0    bat01   
3           3  2022-05-03 06:32:38.127000+00:00      15.5    bat01   
4           4  2022-05-04 11:28:38.085000+00:00       2.6    bat01   

                   DETAILS                                      NAME  \
0  auxiliary_inverter_1_ge  bat01.r01.b.cfo.tgtbqb.dis124.cpt.ea.mes   
1  auxiliary_inverter_1_ge  bat01.r01.b.cfo.tgtbqb.dis124.cpt.ea.mes   
2  auxiliary_inverter_1_ge  bat01.r01.b.cfo.tgtbqb.dis124.cpt.ea.mes   
3  auxiliary_inverter_1_ge  bat01.r01.b.cfo.tgtbqb.dis124.cpt.pa.mes   
4  auxiliary_inverter_1_ge  bat01.r01.b.cfo.tgtbqb.dis124.cpt.pa.mes   

                  SUBCATEGORY           LIBELLE UNITE  
0  active_energy_auxiliary_ge    Energie active   kWh  
1  active_energy_aux

## Question 3.2

In [3]:
x, res, rank, s  = np.linalg.lstsq(A,b)
print("Dimension de A:", A.shape)
print("The least square estimator is:", x)
print("Sum of residuals:", res)
print("Matrix A is of rank:", rank)

C:\Users\maelm\AppData\Local\Temp\ipykernel_24240\4101113602.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x, res, rank, s  = np.linalg.lstsq(A,b)


Dimension de A: (722, 1785)
The least square estimator is: [-0.00927821  0.08309371 -0.03672704 ...  0.01980595 -0.03057174
 -0.01188614]
Sum of residuals: []
Matrix A is of rank: 722


## Question 3.3

In [4]:
diff = np.matmul(A_test,x)-b_test
norm2 = np.linalg.norm(diff)**2
print("Least square difference of train set estimator:", 1/2*norm2)

Least square difference of train set estimator: 140952.17552310223


Since the square of the norm of the difference is this large, maybe the training set is not able to provide a good solution with least square.

## Question 3.5
$f_1 : w \rightarrow \frac{1}{2}||Aw-b||^2+\frac{\lambda}{2}||w||^2$\
$\nabla f_1 = A^T(Aw-b)+\lambda w = (A^TA+\lambda I_n)w-A^Tb$, thus $\nabla^2f_1 = A^TA+\lambda I_n$\
Since $\nabla^2f_1$ is definite positive $f_1$ is strictly convex.

## Question 3.6

In [6]:
lambd = 100
step = 1/np.linalg.norm(np.matmul(A.T,A)+lambd*np.identity(A.shape[1]))

w = np.zeros(A.shape[1])
gradient = np.matmul(A.T,np.matmul(A,w)-b)+lambd*w
iter = 0
while(np.linalg.norm(gradient)>1):
    w = w - step*gradient
    gradient = np.matmul(A.T,np.matmul(A,w)-b)+lambd*w
    iter +=1

print(np.linalg.norm(gradient))
print("Best estimator:",w)
print("Number of iteration needed:", iter)
print("Regularized least square difference of train set estimator:", 1/2*np.linalg.norm(np.matmul(A_test,w)-b_test)**2+lambd/2*np.linalg.norm(w)**2)


C:\Users\maelm\AppData\Local\Temp\ipykernel_24240\1631112457.py:2: RuntimeWarning: invalid value encountered in matmul
  step = 1/np.linalg.norm(np.matmul(A.T,A)+lambd*np.identity(A.shape[1]))


0.9999735973418783
Best estimator: [-0.01238055  0.05775866 -0.00111774 ...  0.01579912 -0.03571617
  0.01335281]
Number of iteration needed: 118766
Regularized least square difference of train set estimator: 54737.17923169638


In [9]:
print("Regularization term:",lambd/2*np.linalg.norm(w)**2)
print("Proportion of the regularization term:",100*lambd/2*np.linalg.norm(w)**2/(1/2*np.linalg.norm(np.matmul(A_test,w)-b_test)**2+lambd/2*np.linalg.norm(w)**2),"%")

Regularization term: 224.30063273329952
Proportion of the regularization term: 0.40977747827278416 %


## Question 4.1
$prox_{g_2}(x) = argmin_y g_2(y)+ \frac {1}{2}||x-y||^2$\
$f_2(w)=\frac {1}{2}||Aw-b||^2$ so $\nabla f_2 = A^T(Aw-b)$

## Question 4.2

In [22]:
lambd = 200
step = 1/np.linalg.norm(np.matmul(A.T,A))
x = np.zeros(A.shape[1])
gradient = np.matmul(A.T,np.matmul(A,x)-b)
iter = 0


def prox(y):
    return np.maximum(np.sign(y)*(np.abs(y)-step*lambd),0)

next = prox(x-step*gradient)
while(np.linalg.norm(x-next)>1e-3):#if x = prox(x-gamma*gradf) then x = argmin f+g
    x = next
    gradient = np.matmul(A.T,np.matmul(A,x)-b)
    iter += 1
    next = prox(x-step*gradient)
    print(iter)
    print(np.linalg.norm(x-next))

print("Best estimator:",x)
print("Number of iteration needed:", iter)
print("Regularized least square difference of train set estimator obtained from lasso problem:",1/2*np.linalg.norm(np.matmul(A_test,x)-b_test)**2+lambd*np.linalg.norm(x,1))

C:\Users\maelm\AppData\Local\Temp\ipykernel_24240\218417930.py:2: RuntimeWarning: invalid value encountered in matmul
  step = 1/np.linalg.norm(np.matmul(A.T,A))


1
0.011110519438794314
2
0.007097333324298534
3
0.005097138677407172
4
0.0038546551023445346
5
0.0031141658426448674
6
0.00270523939591563
7
0.0024431554955535568
8
0.0022493722082329317
9
0.0021334668430295035
10
0.002026825880580864
11
0.0019453523897822508
12
0.0019024755836914832
13
0.0018587152728220596
14
0.0018192119759440158
15
0.0017787354884999877
16
0.0017516154182213352
17
0.0017191151056098405
18
0.001695235775257057
19
0.0016725018463676557
20
0.0016574392080668428
21
0.0016410452992265563
22
0.0016318572636036704
23
0.0016149414613021403
24
0.0016071046781523644
25
0.0015987238890863697
26
0.0015862138109430225
27
0.0015778486485501036
28
0.0015751926372849197
29
0.0015631120075121855
30
0.0015560731485806576
31
0.0015446237244254126
32
0.0015365296246529325
33
0.0015282150392949344
34
0.0015204700465105653
35
0.0015091309283447821
36
0.0014906366380010604
37
0.0014961003584801449
38
0.0014900588611331592
39
0.0014769996094254301
40
0.0014691914643377528
41
0.00146226877

In [24]:
print(np.linalg.norm(x-next))
print("Regularization term:",lambd*np.linalg.norm(x,1))
print("Proportion of the regularization term:",100*lambd*np.linalg.norm(x,1)/(1/2*np.linalg.norm(np.matmul(A_test,x)-b_test)**2+lambd*np.linalg.norm(x,1)),"%")

0.0009999729170850416
Regularization term: 683.4944121677574
Proportion of the regularization term: 23.9306873981019 %


# Question 4.3

In [25]:
lambd = 200
b = 0.5
t=1
x = np.zeros(A.shape[1])
gradient = np.matmul(A.T,np.matmul(A,x)-b)
iter = 0


def prox(y):
    return np.maximum(np.sign(y)*(np.abs(y)-step*lambd),0)

xplus = prox(x-t*gradient)
while(1/2*np.linalg.norm(np.matmul(A,xplus)-b)**2> 1/2*np.linalg.norm(np.matmul(A,x)-b)**2+np.dot(gradient,xplus-x)+1/(2*t)*np.linalg.norm(xplus-x)**2):
        t = b*t
        xplus = prox(x-t*gradient)

while(np.linalg.norm(x-xplus)>1e-3):#if x = prox(x-gamma*gradf) then x = argmin f+g
    t=1
    x = xplus
    gradient = np.matmul(A.T,np.matmul(A,x)-b)
    iter += 1
    while(1/2*np.linalg.norm(np.matmul(A,xplus)-b)**2> 1/2*np.linalg.norm(np.matmul(A,x)-b)**2+np.dot(gradient,xplus-x)+1/(2*t)*np.linalg.norm(xplus-x)**2):
        t = b*t
        xplus = prox(x-t*gradient)
    print(iter)
    print(np.linalg.norm(x-xplus))

print("Best estimator:",x)
print("Number of iteration needed:", iter)
print("Regularized least square difference of train set estimator obtained from lasso problem:",1/2*np.linalg.norm(np.matmul(A_test,x)-b_test)**2+lambd*np.linalg.norm(x,1))

1
0.0
Best estimator: [0.00000000e+00 4.70357782e-05 4.70357782e-05 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
Number of iteration needed: 1
Regularized least square difference of train set estimator obtained from lasso problem: 13224.055048960341
